In [1]:
# For data manipulation
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import VotingClassifier
# For plotting
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('darkgrid')

In [2]:
df = pd.read_csv('spy_generated_features.csv',index_col=0)
x = df.filter(like='f_')
y = df['target']
x

,f_ret_1,f_ret_2,f_ret_3,f_ret_5,f_ret_14,f_ret_21,f_ret_50,f_natr,f_rsi,f_norm_upper,f_norm_lower,f_norm_middle
Date,,,,,,,,,,,,
2009-03-18,0.022384,0.053651,0.050467,0.100358,0.056995,0.008963,-0.139149,3.694927,57.365335,1.007049,0.922440,0.964744
2009-03-19,-0.012386,0.009721,0.040601,0.045563,0.067767,-0.001139,-0.155451,3.683051,54.703675,1.025887,0.945230,0.985559
2009-03-20,-0.021294,-0.033416,-0.011780,0.015364,0.094321,-0.011780,-0.147909,3.716001,50.425422,1.046203,0.973853,1.010028
2009-03-23,0.071829,0.049005,0.036013,0.091596,0.181796,0.069601,-0.090417,3.704093,61.210804,1.005176,0.913069,0.959123
2009-03-24,-0.019703,0.050711,0.028337,0.038334,0.131701,0.087433,-0.088821,3.672594,57.266114,1.032415,0.939152,0.985784
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-22,-0.001813,0.023315,0.025521,0.005717,-0.081991,-0.034963,-0.159892,2.666927,38.480883,1.019872,0.965183,0.992528
2022-06-23,0.009803,0.007972,0.033346,0.001300,-0.090316,-0.043401,-0.136904,2.564619,41.207044,1.010850,0.955454,0.983152
2022-06-24,0.031794,0.041908,0.040019,0.068499,-0.045732,-0.005395,-0.106151,2.528335,49.154390,1.005813,0.925544,0.965679


In [3]:
y

Date
2009-03-18    0
2009-03-19    0
2009-03-20    1
2009-03-23    0
2009-03-24    1
             ..
2022-06-22    1
2022-06-23    1
2022-06-24    0
2022-06-27    0
2022-06-28    0
Name: target, Length: 3344, dtype: int64

In [4]:
split_percentage = 0.8
split = int(split_percentage*len(x))

# Train dataset
x_train = x[:split]
y_train = y[:split]

# Test dataset
x_test = x[split:]
y_test = y[split:]

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(2675, 12) (2675,)
(669, 12) (669,)


In [5]:
xgb = XGBClassifier(max_depth=5, n_estimators=50,
                    random_state=42)
lr = LogisticRegression(random_state=42)
svc = svm.SVC(kernel='rbf', probability=True, random_state=42)

estimators = []
estimators.append(('LR', lr))
estimators.append(('SVC', svc))
estimators.append(('XGB', xgb))

In [6]:
vot_soft = VotingClassifier(estimators=estimators, voting='hard')
vot_soft.fit(x_train, y_train)

VotingClassifier(estimators=[('LR', LogisticRegression(random_state=42)),
                             ('SVC', SVC(probability=True, random_state=42)),
                             ('XGB',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            gpu_id=None, grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=5,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=50, n_jobs=None,
                                            num_parallel_tree=None,
                                            predictor=None, random_state=42, ...))])

In [7]:
from sklearn.metrics import accuracy_score

y_pred = vot_soft.predict(x_test)
score = accuracy_score(y_test, y_pred)
f"Test Accuracy Score {score}"

'Test Accuracy Score 0.5605381165919282'

In [8]:
import pickle
pickle.dump(vot_soft, open('vot_soft.pickle', "wb"))